In [1]:
!pip install jupyterlab-vim
!jupytexjupyter labextension enable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.8 MB/s eta 0:00:00
/usr/bin/sh: 1: jupytexjupyter: not found


# Neo4j

In [2]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 4.9 MB/s eta 0:00:00a 0:00:01


In [13]:
#!curl http://neo4j:7474
#!curl http://neo4j:7687

In [37]:
from neo4j import GraphDatabase, RoutingControl

#URI = "neo4j://localhost:7687"
#URI = "bolt://localhost:7687"
#URI = "bolt://127.0.0.1:7687"
URI = "bolt://neo4j:7687"
AUTH = ("neo4j", "testtest")

def add_friend(driver, name, friend_name):
    driver.execute_query(
        "MERGE (a:Person {name: $name}) "
        "MERGE (friend:Person {name: $friend_name}) "
        "MERGE (a)-[:KNOWS]->(friend)",
        name=name, friend_name=friend_name, database_="neo4j",
    )


def print_friends(driver, name):
    records, _, _ = driver.execute_query(
        "MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
        "RETURN friend.name ORDER BY friend.name",
        name=name, database_="neo4j", routing_=RoutingControl.READ,
    )
    for record in records:
        print(record["friend.name"])


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    add_friend(driver, "Arthur", "Guinevere")
    add_friend(driver, "Arthur", "Lancelot")
    add_friend(driver, "Arthur", "Merlin")
    print_friends(driver, "Arthur")

Guinevere
Lancelot
Merlin


# Example

In [39]:
driver = GraphDatabase.driver(URI, auth=AUTH)

- Every application using Neo4j needs a `driver` object
- A `driver` object holds the details to the connection to a Neo4j database (e.g., URIs, credentials, and configuration).

In [95]:
# Test the connection to the DB without executing any query.
driver.verify_connectivity()

In [96]:
driver.get_server_info()

In [98]:
# `execute_query()` wraps lower level APIs (e.g., Sessions) and it's used for simple cases.
query = "MATCH(n) RETURN COUNT(n) AS node_count"
query_result = driver.execute_query(query)

In [86]:
print("type=", type(query_result))
print(query_result)

type= <class 'neo4j._work.eager_result.EagerResult'>
EagerResult(records=[<Record node_count=1>], summary=<neo4j._work.summary.ResultSummary object at 0xffff3f381d20>, keys=['node_count'])


In [89]:
results, summary, keys = query_result
print(records)
print(summary)
print(keys)

[<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>]
['node_count']


## Session

- Database activity is coordinated through `Session`s and `Transaction`s
- A `Session` is a container for a number of unit of works
    - Provide guarantees of causal consistency
    - Are lightweight opeation and not thread safe
- A `Transaction` is a unit of work that is either committed in its entirety or rolled back in case of failure

In [105]:
# Create `Session`.
session = driver.session(database="neo4j")
print("session.closed()=", session.closed())

# Run a query.
query = "MATCH (n) RETURN n"
_ = session.run(query)

# Close `Session`.
session.close()
print("session.closed()=", session.closed())

session.closed()= False
session.closed()= True


In [99]:
# Session can be created and destroyed using a block context, so that the session is closed
# properly in case of exceptions.
with driver.session() as session:
    result = session.run("MATCH (n) RETURN n")
    # ...

In [62]:
# Count the number of nodes.
query = "MATCH(n) RETURN COUNT(n) AS node_count"
node_count_result = session.run(query)
node_count = node_count_result.single()["node_count"]
print(node_count)

1


In [61]:
# Count the number of relations.
query = "MATCH ()-[r]->() RETURN COUNT(r) AS relationship_count;"
result = session.run(query)
rel_count = result.single()["relationship_count"]
print(rel_count)

0


In [70]:
query = "MATCH (n) RETURN n"
records, summary, keys = driver.execute_query(query)
print(records)
print(summary)
print(keys)

[<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>]
['n']


In [71]:
for record in records:
    print(record.data())

{'n': {'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}}


In [75]:
print(summary)

In [73]:
# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

The query `MATCH (n) RETURN n` returned 1 records in 27 ms.


In [76]:
print(records[0])

<Record n=<Node element_id='4:907b90c5-77b7-40ee-bd2b-900a55534cf9:5' labels=frozenset({'Wine'}) properties={'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}>>


In [77]:
print(records[0].data())

{'n': {'vintage': 2015, 'name': 'Prancing Wolf', 'style': 'ice wine'}}


In [52]:
# node_count_result = driver.execute_query(
#     "MATCH(n) RETURN COUNT(n) AS node_count")
# node_count = node_count_result["node_count"]

TypeError: tuple indices must be integers or slices, not str

In [56]:
# Delete everything.
driver.execute_query(
    "MATCH(n) OPTIONAL MATCH (n)-[r]-() DELETE n, r"
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff3f604100>, keys=[])

In [58]:
query = 'CREATE (w:Wine {name:"Prancing Wolf", style: "ice wine", vintage: 2015})'
driver.execute_query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0xffff3f48e950>, keys=[])

In [ ]:
driver.execute_query(
    "MERGE (a:Person {name: $name}) "
    "MERGE (friend:Person {name: $friend_name}) "
    "MERGE (a)-[:KNOWS]->(friend)",
    name=name, friend_name=friend_name, database_="neo4j",
)

In [ ]:
session.close()
driver.close()

# pyvis

In [23]:
!pip install --quiet pyvis

In [36]:
from pyvis.network import Network

# Sample Cypher query to retrieve nodes and relationships
cypher_query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 10
"""
# Execute the query and fetch the results

with driver.session() as session:
  result = session.run(cypher_query)

  net = Network(cdn_resources = "remote", directed = True, height = '500px',width = '100%',
                  notebook = True)

  for record in result:
    #print(result)
    node_a = record["n"]
    node_b = record["m"]
    relationship = record["r"]

    #add nodes
    net.add_node(node_a.element_id)
    net.add_node(node_b.element_id)
    net.add_edge(node_a.element_id, node_b.element_id)

#save html format
#net.show(file_name,notebook=False)

net.show("ex.html")

ex.html


/tmp/ipykernel_35/186928212.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


# Langchain

In [7]:
!pip install langchain
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 9.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 651.9/651.9 kB 9.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00:00:010:01


In [16]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

In [9]:
from langchain.graphs import Neo4jGraph

username ="neo4j"
password = "testtest"
graph = Neo4jGraph(
    url=URI,
    username=username,
    password=password
)

In [14]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = URI
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "testtest"

graph = Neo4jGraph()

In [21]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# Import movie information

movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

In [19]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

In [20]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Institution'), Node(id='Nobel Prize', type='Award')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Nobel Prize', type='Award'), type='WINNER'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='University Of Paris', type='Institution'), type='PROFESSOR'), Relationship(source=Node(id='Pierre Curie', type='Person'), target=Node(id='Nobel Prize', type='Award'), type='WINNER')]
